<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2/DACON_vision2_0213_1_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 실행시 등장하는 URL을 클릭하여 허용해주면 인증KEY가 나타난다. 복사하여 URL아래 빈칸에 붙여넣으면 마운트에 성공하게된다.
from google.colab import drive
drive.mount('./MyDrive')

Drive already mounted at ./MyDrive; to attempt to forcibly remount, call drive.mount("./MyDrive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


######################################################
# File Load
dataset = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/dirty_mnist_2nd_answer.csv')
y_df =  dataset.iloc[:,:]
print(y_df.shape)  # (50000, 27)

sub = pd.read_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sample_submission.csv')
print(sub.shape)    # (5000, 27)

######################################################

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    fill_mode='nearest'
)
etc_datagen = ImageDataGenerator(rescale=1./255)

def mymodel () :
    model = Sequential()
    model.add(Conv2D(32, (2,2), padding='same', input_shape=(80, 80, 1), activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(64, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (2,2), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization()) 
    model.add(Dense(1, activation='sigmoid'))

    return model

#1. DATA
x = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x4.npy')
x_pred = np.load('./MyDrive/MyDrive/Colab/DACON_vision2/npy/vision_x_pred4.npy')
print("<==complete load==>")

print(x.shape, x_pred.shape) # (50000, 80, 80, 1) (5000, 80, 80, 1)
x[x < 253] = 0

alphabet = ['b']

val_loss_list = []
val_acc_list = []

for alph in alphabet :  # 한 알파벳씩 검증
    print('<<<<<<<<<<<< ', alph, ' predict start >>>>>>>>>>>>')
    y = y_df.loc[:,alph]
    y_pred_list = []
    y_result = 0

    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=42)
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, train_size=0.8, shuffle=True, random_state=42)
    print(x_train.shape, x_test.shape, x_valid.shape)  # (32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
    print(y_train.shape, y_test.shape, y_valid.shape)  # (32000,) (10000,) (8000,)

    batch = 40
    train_generator = train_datagen.flow(x_train, y_train, batch_size=batch, seed=2021)
    test_generator = etc_datagen.flow(x_test, y_test, batch_size=batch, seed=2021)
    valid_generator = etc_datagen.flow(x_valid, y_valid)
    pred_generator = etc_datagen.flow(x_pred)

    #2. Modeling
    model = mymodel()

    #3. Compile, Train

    path = './MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_b.hdf5'
    cp = ModelCheckpoint(filepath=path, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    es = EarlyStopping(monitor='val_loss', patience=30, mode='min')
    lr = ReduceLROnPlateau(monitor='val_loss', patience=15, factor=0.4, mode='min')

    model.compile(
        loss='binary_crossentropy', 
        optimizer=Adam(lr=0.05, epsilon=None), 
        metrics=['acc']
    )

    hist = model.fit_generator(
        train_generator, epochs=50,
        steps_per_epoch = len(x_train) // batch ,
        validation_data=valid_generator, callbacks=[es, lr, cp]
    )

    # loss가 가장 작은 것 로드
    model = load_model('./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_b.hdf5')

    #4. Evaluate, Predict
    loss, acc = model.evaluate(test_generator)
    print("loss : ", loss)
    print("acc : ", acc)  

    val_loss_list.append(loss)
    val_acc_list.append(acc)

    # loss :  
    # acc :  

    y_pred = model.predict(pred_generator)
    # print(y_pred.shape) # (5000, 1)
    y_result= np.where(y_pred < 0.5, 0, 1)
    sub.loc[:,alph] = y_result  # 예측값을 sub 파일에 저장
    print(sub.head())

    print("- ", alph, " predict end -\n")

print("mean loss : ", sum(val_loss_list)/len(alphabet))
print("mean acc : ", sum(val_acc_list)/len(alphabet))

sub.to_csv('./MyDrive/MyDrive/Colab/DACON_vision2/sub_0213_1_b.csv', index=False)
print(sub.head())

# sub
# score 

(50000, 27)
(5000, 27)
<==complete load==>
(50000, 80, 80, 1) (5000, 80, 80, 1)
<<<<<<<<<<<<  b  predict start >>>>>>>>>>>>
(32000, 80, 80, 1) (10000, 80, 80, 1) (8000, 80, 80, 1)
(32000,) (10000,) (8000,)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
800/800 [==============================] - 27s 31ms/step - loss: 0.7133 - acc: 0.5228 - val_loss: 0.6937 - val_acc: 0.5394

Epoch 00001: val_loss improved from inf to 0.69374, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_b.hdf5
Epoch 2/50
800/800 [==============================] - 25s 31ms/step - loss: 0.6937 - acc: 0.5265 - val_loss: 0.6914 - val_acc: 0.5394

Epoch 00002: val_loss improved from 0.69374 to 0.69141, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_b.hdf5
Epoch 3/50
800/800 [==============================] - 24s 31ms/step - loss: 0.6940 - acc: 0.5288 - val_loss: 0.6946 - val_acc: 0.5063

Epoch 00003: val_loss did not improve from 0.69141
Epoch 4/50
800/800 [==============================] - 25s 31ms/step - loss: 0.6936 - acc: 0.5300 - val_loss: 0.6902 - val_acc: 0.5394

Epoch 00004: val_loss improved from 0.69141 to 0.69018, saving model to ./MyDrive/MyDrive/Colab/DACON_vision2/cp/vision_0213_1_b.hdf5
Epoch 5/50


In [ ]:
# b